In [3]:
from AC import preproc
from AC import get_news
import inspect
import nltk
nltk.download("stopwords")

%load_ext autoreload
%autoreload 2

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Cristy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
economia = get_news("./articoli_economia/")
cultura = get_news("./articoli_cultura/")
tech = get_news("./articoli_tech/")
politica = get_news("./articoli_politica/")
sport = get_news("./articoli_sport/")
cronaca = get_news("./articoli_cronaca/")

for articolo in economia:
    articolo['categoria'] = "Economia"
for articolo in cultura:
    articolo['categoria'] = "Cultura"
for articolo in tech:
    articolo['categoria'] = "Tech"
for articolo in politica:
    articolo['categoria'] = "Politica"
for articolo in sport:
    articolo['categoria'] = "Sport"
for articolo in cronaca:
    articolo['categoria'] = "Cronaca"
dati_preprocessati =  preproc(tech + politica + cultura + economia + sport + cronaca)

100%|██████████| 2400/2400 [00:15<00:00, 156.22it/s]


In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import ParameterGrid

import numpy as np
import pandas as pd
import math

import tqdm
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn import linear_model,metrics

In [4]:
seed = 0
train_documents,test_documents = train_test_split(dati_preprocessati,random_state=seed)
test_documents, val_documents = train_test_split(test_documents,random_state=seed,train_size=0.5)

C:\Users\Cristy\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [5]:
docs_texts = [' '.join([word for word in x['testo']] + x['tags'] + x['sottotitolo'] + x['titolo_articolo']) for x in dati_preprocessati]
train_texts = [' '.join([word for word in x['testo']] + x['sottotitolo'] + x['titolo_articolo']) for x in train_documents]
test_texts = [' '.join([word for word in x['testo']] + x['sottotitolo'] + x['titolo_articolo']) for x in test_documents]
val_texts = [' '.join([word for word in x['testo']] + x['sottotitolo'] + x['titolo_articolo']) for x in val_documents]

docs_cats = [x["categoria"] for x in dati_preprocessati]
train_cats = [x["categoria"] for x in train_documents]
test_cats = [x["categoria"] for x in test_documents]
val_cats = [x["categoria"] for x in val_documents]

# Ottimizzazione con LDA

In [1]:
ldac = Pipeline([
    ("count_mx",CountVectorizer(max_features=1000000, lowercase=True)),
    ("lda", LatentDirichletAllocation(max_iter=100, learning_method='online',random_state=0)),
    ("classifier",DecisionTreeClassifier(random_state=0))
])

params = {
    'count_mx__ngram_range': (1, 3),
    'count_mx__min_df': 10,
    'count_mx__max_df': 200,
    'lda__n_components':48,
    'lda__learning_decay':0.7,
    'classifier__min_samples_leaf': 1,
    'classifier__max_depth': 20
}
ldac.set_params(**params)

NameError: name 'Pipeline' is not defined

In [41]:
n = ldac.named_steps["classifier"].max_depth
print("Profondità dell'albero allenato senza restrizioni: {}".format(ldac.named_steps["classifier"].max_depth))
print("Massimo numero minimo di osservazioni in una foglia: {}".format(len(train_documents) // 2))

Profondità dell'albero allenato senza restrizioni: 20
Massimo numero minimo di osservazioni in una foglia: 900


In [43]:
params_tdm = {
    'ngram_range': (1, 3),
    'min_df': 10,
    'max_df': 200
}

params_lda = {
    'n_components':48,
    'learning_decay':0.7    
}

tf = CountVectorizer(max_features=1000000, lowercase=True, **params_tdm).fit(train_texts)
tf_train = tf.transform(train_texts)
ldax = LatentDirichletAllocation(max_iter=50, learning_method='online', random_state=0, **params_lda).fit(tf_train.toarray())
ldax_train = ldax.transform(tf_train.toarray())
tf_val = tf.transform(val_texts)
ldax_val = ldax.transform(tf_val.toarray())

param_grid = ParameterGrid({
    'max_depth': np.arange(1, n+5),
    'min_samples_leaf': 2 ** np.arange(int(math.log(len(train_documents) // 2,2))),
})
print(param_grid.param_grid)

[{'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24]), 'min_samples_leaf': array([  1,   2,   4,   8,  16,  32,  64, 128, 256], dtype=int32)}]


In [44]:
risultati = []
for params in tqdm.tqdm(param_grid):
    ldac = DecisionTreeClassifier(random_state=0, **params).fit(ldax_train, train_cats)
    y_pred = ldac.predict(ldax_val)
    params["accuracy_score"] = metrics.accuracy_score(val_cats, y_pred)
    risultati.append(params)

risultati = pd.DataFrame(risultati).sort_values(["accuracy_score", "max_depth"], ascending=[False, True])
risultati.reset_index(drop=True, inplace=True)
print("Primi 5:")
display(risultati.head())

print("Ultimi 5:")
risultati.tail()

100%|██████████| 216/216 [00:03<00:00, 58.01it/s]


Primi 5:


,accuracy_score,max_depth,min_samples_leaf
0,0.983333,22,1
1,0.983333,23,1
2,0.983333,24,1
3,0.976667,22,2
4,0.976667,23,2


Ultimi 5:


,accuracy_score,max_depth,min_samples_leaf
211,0.273333,1,8
212,0.273333,1,16
213,0.273333,1,32
214,0.273333,1,64
215,0.273333,1,128


In [ ]:
# non credo valga la pena vedere come fare la visualizzazione delle cose -_-

In [ ]:
# eventualmente effettuare ottimizzazione per un altro numero di componenti dell'LDA

# Ottimizzazione senza LDA

In [9]:
tfc = Pipeline([
    ("classifier",DecisionTreeClassifier(random_state=0))
])

params_tdm = {
    'ngram_range': (1, 1),
    'min_df': 6,
    'max_df': 0.5
}

tf = CountVectorizer(max_features=1000000, lowercase=True, **params_tdm).fit(train_texts)
tf_train = tf.transform(train_texts)
tf_val = tf.transform(val_texts)

In [ ]:
print("Profondità dell'albero allenato senza restrizioni: {}".format(ldac.named_steps["classifier"].max_depth))
print("Massimo numero minimo di osservazioni in una foglia: {}".format(train_documents // 2))

In [ ]:
param_grid = ParameterGrid({
    'classifier__max_depth': np.arange(1, ldac.named_steps["classifier"].max_depth+5),
    'classifier__min_samples_leaf': 2 ** np.arange(int(math.log(train_documents // 2,2))),
})
print(param_grid.param_grid)

In [ ]:
risultati = []



for params in tqdm.tqdm(param_grid):
    tfc.set_params(**params)
    tfc.fit(tf_train.toarray(), train_cats)#verificare se sostituibile con 
    #tfc = DecisionTreeClassifier(random_state=0, **params).fit(tf_train.toarray(), train_cats)
    y_pred = tfc.predict(tf_val.toarray())
    params["accuracy_score"] = metrics.accuracy_score(val_cats, y_pred)
    risultati.append(params)

risultati = pd.DataFrame(risultati).sort_values(["accuracy_score", "classifier__max_depth"], ascending=[False, True])
risultati.reset_index(drop=True, inplace=True)
print("Primi 5:")
display(risultati.head())

print("Ultimi 5:")
risultati.tail()